# Word tagging exercice


In [38]:
print(keras.__version__)

1.1.2


In [9]:
import numpy as np 

import pickle

atis_file = '/Users/jorge/data/training/text/atis/atis.pkl'
with open(atis_file,'rb') as f:
    train, test, dicts = pickle.load(f, encoding='bytes')


In [14]:
print(dicts.keys())

dict_keys([b'tables2idx', b'words2idx', b'labels2idx'])


## train / test sets:
    - X: list of sequences
    - ne: PENDING
    - label: name entities
## Dictionaries
    - labels2idx
    - words2idx

In [17]:
# Visualize data

#w2idx, ne2idx, labels2idx = dicts['words2idx'], dicts['tables2idx'], dicts['labels2idx']
#idx2w  = dict((v,k) for k,v in w2idx.iteritems())
#idx2ne = dict((v,k) for k,v in ne2idx.iteritems())
#idx2la = dict((v,k) for k,v in labels2idx.iteritems())


w2idx, ne2idx, labels2idx = dicts[b'words2idx'], dicts[b'tables2idx'], dicts[b'labels2idx']
    
idx2w  = dict((v,k) for k,v in w2idx.items())
idx2ne = dict((v,k) for k,v in ne2idx.items())
idx2la = dict((v,k) for k,v in labels2idx.items())

train_x, train_ne, train_label = train
test_x,  test_ne,  test_label  = test
wlength = 35

for e in ['train','test']:
    print(e)
    for sw, se, sl in zip(eval(e+'_x')[:5], eval(e+'_ne')[:5], eval(e+'_label')[:10]):
        print( 'WORD'.rjust(wlength), 'LABEL'.rjust(wlength))
        for wx, la in zip(sw, sl): print( idx2w[wx].rjust(wlength), idx2la[la].rjust(wlength))
        print( '\n'+'**'*30+'\n')


train
                               WORD                               LABEL
b'                                  i' b'                                  O'
b'                               want' b'                                  O'
b'                                 to' b'                                  O'
b'                                fly' b'                                  O'
b'                               from' b'                                  O'
b'                             boston' b'                B-fromloc.city_name'
b'                                 at' b'                                  O'
b'                    DIGITDIGITDIGIT' b'                 B-depart_time.time'
b'                                 am' b'                 I-depart_time.time'
b'                                and' b'                                  O'
b'                             arrive' b'                                  O'
b'                                 in' b'                       

# Model

For each word in the sentence, predict the label

First, for each word in each sentence, generate  the context with the previous and next words in the sentence.
This create a train dataset of num_sentences*

In [18]:
def context(l, size=3):
    l = list(l)
    lpadded = size // 2 * [572] + l + size // 2 * [572]
    out = [lpadded[i:(i + size)] for i in range(len(l))]
    return out

x = np.array([0, 1, 2, 3, 4], dtype=np.int32)
context(x)

[[572, 0, 1], [0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 572]]

In [92]:
3 / 2

1

In [19]:
X_trn=[]
for s in train_x:
    X_trn += context(s,size=10)
X_trn = np.array(X_trn)

X_tst=[]
for s in test_x:
    X_tst += context(s,size=10)
X_tst = np.array(X_tst)

print(X_trn.shape, X_tst.shape)

(56590, 10) (9198, 10)


In [20]:
y_trn=[]
for s in train_label:
    y_trn += list(s)
y_trn = np.array(y_trn)
print(y_trn.shape)

y_tst=[]
for s in test_label:
    y_tst += list(s)
y_tst = np.array(y_tst)
print(y_tst.shape)

(56590,)
(9198,)


In [21]:
print(len(set(y_trn)))
print(len(set(idx2w)))

121
572


In [26]:
#Model parameters

input_seq_length = X_trn.shape[1]
input_vocabulary_size = len(set(idx2w)) + 1
output_length = 127
num_hidden_lstm = 128

In [27]:
# build the model: Simple LSTM with embedings
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Embedding

print('Build model 1')
seq_input = Input(shape=([input_seq_length]), name='prev') 
    
embeds = Embedding(input_vocabulary_size, 64)(seq_input)
    
# apply forwards LSTM
#forwards1 = LSTM(512, dropout_W=0.3, dropout_U=0.3, return_sequences=True)(embed)

forwards = LSTM(num_hidden_lstm, return_sequences=False)(embeds)

output = Dense(output_length, activation='softmax')(forwards)

model1 = Model(input=seq_input, output=output)
model1.summary()

# Optimizer
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Build model 1
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
prev (InputLayer)                (None, 10)            0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 10, 64)        36672       prev[0][0]                       
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 128)           98816       embedding_2[0][0]                
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 127)           16383       lstm_2[0][0]                     
Total params: 151871
________________________________________________________

In [28]:
#Print the model
#Plot the model graph
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(model1).create(prog='dot', format='svg'))

ImportError: No module named 'pydot'

In [29]:
#Fit model
history = model1.fit(X_trn, y_trn, batch_size=128, nb_epoch=10,
           validation_data=(X_tst, y_tst))


Train on 56590 samples, validate on 9198 samples
Epoch 1/10
56590/56590 [==============================] - 23s - loss: 1.5154 - acc: 0.6893 - val_loss: 1.0233 - val_acc: 0.7595
Epoch 2/10
56590/56590 [==============================] - 23s - loss: 0.6655 - acc: 0.8387 - val_loss: 0.5498 - val_acc: 0.8776
Epoch 3/10
56590/56590 [==============================] - 23s - loss: 0.3561 - acc: 0.9169 - val_loss: 0.3727 - val_acc: 0.9110
Epoch 4/10
56590/56590 [==============================] - 24s - loss: 0.2172 - acc: 0.9500 - val_loss: 0.2943 - val_acc: 0.9376
Epoch 5/10
56590/56590 [==============================] - 23s - loss: 0.1529 - acc: 0.9657 - val_loss: 0.2396 - val_acc: 0.9497
Epoch 6/10
56590/56590 [==============================] - 23s - loss: 0.1130 - acc: 0.9756 - val_loss: 0.1994 - val_acc: 0.9611
Epoch 7/10
56590/56590 [==============================] - 23s - loss: 0.0911 - acc: 0.9802 - val_loss: 0.1801 - val_acc: 0.9634
Epoch 8/10
56590/56590 [==============================]

1.1.2


In [34]:
# Bidirectional
# build the model: Simple LSTM with embedings
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Embedding, Merge, Bidirectional

print('Build model 2')
seq_input = Input(shape=([input_seq_length]), name='prev') 
    
embeds = Embedding(input_vocabulary_size, 64)(seq_input)
    
# Bidirectional wrapper
rnn_out = Bidirectional(LSTM(num_hidden_lstm))(embeds)

output = Dense(output_length, activation='softmax')(rnn_out)

model2 = Model(input=seq_input, output=output)
model2.summary()

# Optimizer
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Build model 2
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
prev (InputLayer)                (None, 10)            0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 10, 64)        36672       prev[0][0]                       
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 256)           197632      embedding_5[0][0]                
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 127)           32639       bidirectional_1[0][0]            
Total params: 266943
________________________________________________________

In [39]:
#Fit model
history2 = model2.fit(X_trn, y_trn, batch_size=128, nb_epoch=10,
           validation_data=(X_tst, y_tst))


Train on 56590 samples, validate on 9198 samples
Epoch 1/10
56590/56590 [==============================] - 51s - loss: 1.3319 - acc: 0.7182 - val_loss: 0.7190 - val_acc: 0.8200
Epoch 2/10
56590/56590 [==============================] - 50s - loss: 0.3801 - acc: 0.9043 - val_loss: 0.3315 - val_acc: 0.9213
Epoch 3/10
56590/56590 [==============================] - 52s - loss: 0.1627 - acc: 0.9610 - val_loss: 0.2157 - val_acc: 0.9572
Epoch 4/10
56590/56590 [==============================] - 52s - loss: 0.0952 - acc: 0.9773 - val_loss: 0.1759 - val_acc: 0.9653
Epoch 5/10
56590/56590 [==============================] - 52s - loss: 0.0682 - acc: 0.9833 - val_loss: 0.1738 - val_acc: 0.9649
Epoch 6/10
56590/56590 [==============================] - 52s - loss: 0.0524 - acc: 0.9872 - val_loss: 0.1892 - val_acc: 0.9624
Epoch 7/10
56590/56590 [==============================] - 52s - loss: 0.0434 - acc: 0.9885 - val_loss: 0.1776 - val_acc: 0.9669
Epoch 8/10
56590/56590 [==============================]